# Task 1: Binary Representations

This section implements fundamental bitwise operations used in cryptographic functions.
We define:
- **rotl(x, n)**: Left-rotates a 32-bit integer `x` by `n` places.
- **rotr(x, n)**: Right-rotates a 32-bit integer `x` by `n` places.
- **ch(x, y, z)**: Chooses bits based on `x` (from `y` if `x` has 1s, from `z` otherwise).
- **maj(x, y, z)**: Computes the majority vote of the bits from `x, y, z`.

In [32]:
def rotl(x, n=1):
    """
    Rotates the bits in a 32-bit unsigned integer to the left by n places.
    
    Args:
    - x (int): A 32-bit integer.
    - n (int): Number of positions to rotate.
    
    Returns:
    - int: The rotated integer.
    """
    n = n % 32  # Ensure n is within 0-31
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF  # Mask to 32 bits

def rotr(x, n=1):
    """
    Rotates the bits in a 32-bit unsigned integer to the right by n places.
    """
    n = n % 32
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

def ch(x, y, z):
    """
    Chooses bits from y where x has bits set to 1 and from z where x has bits set to 0.
    """
    return (x & y) ^ (~x & z)

def maj(x, y, z):
    """
    Takes a majority vote of the bits in x, y, and z.
    """
    return (x & y) ^ (x & z) ^ (y & z)


Testing the Bitwise Functions:
We use three test values:
- **x = 0xB3333333**
- **y = 0xCCCCCCCC**
- **z = 0xF0F0F0F0**

We will rotate `x`, apply `ch(x, y, z)`, and compute `maj(x, y, z)`.


In [33]:
# Test values
test_x = 0b10110011001100110011001100110011  # 0xB3333333
test_y = 0b11001100110011001100110011001100  # 0xCCCCCCCC
test_z = 0b11110000111100001111000011110000  # 0xF0F0F0F0

# Print the results
print(f"rotl({test_x:032b}, 4)  -> {rotl(test_x, 4):032b}")
print(f"rotr({test_x:032b}, 4)  -> {rotr(test_x, 4):032b}")
print(f"ch({test_x:032b}, {test_y:032b}, {test_z:032b}) -> {ch(test_x, test_y, test_z):032b}")
print(f"maj({test_x:032b}, {test_y:032b}, {test_z:032b}) -> {maj(test_x, test_y, test_z):032b}")


rotl(10110011001100110011001100110011, 4)  -> 00110011001100110011001100111011
rotr(10110011001100110011001100110011, 4)  -> 00111011001100110011001100110011
ch(10110011001100110011001100110011, 11001100110011001100110011001100, 11110000111100001111000011110000) -> 11000000110000001100000011000000
maj(10110011001100110011001100110011, 11001100110011001100110011001100, 11110000111100001111000011110000) -> 11110000111100001111000011110000


# Task 2: Hash Functions

This function computes a simple hash value for a string using the following logic:
1. Initialize `hashval = 0`.
2. Iterate over each character in the string.
3. Compute `hashval = ord(char) + 31 * hashval`.
4. Take the final value modulo 101.

This method ensures good hash distribution and minimizes collisions.


In [34]:
def hash_function(s: str) -> int:
    """
    Computes a simple hash value for a string using a rolling hash approach.
    
    Args:
    - s (str): The input string.
    
    Returns:
    - int: The hash value modulo 101.
    """
    hashval = 0
    for char in s:
        hashval = ord(char) + 31 * hashval  # Hash function using 31
    return hashval % 101  # Modulo 101 to limit hash size


Testing the Hash Function:
We test the function with different strings to observe the hash values.


In [35]:
# Test cases
test_strings = ["hello", "world", "hashing", "kernighan", "ritchie"]

# Compute hashes
for s in test_strings:
    print(f"Hash of '{s}': {hash_function(s)}")


Hash of 'hello': 17
Hash of 'world': 34
Hash of 'hashing': 25
Hash of 'kernighan': 37
Hash of 'ritchie': 26


The values 31 and 101 may be used in this hash function due to:

31 is a prime number, which helps in generating a more uniform distribution of [hash values](https://www.geeksforgeeks.org/why-does-javas-hashcode-in-string-use-31-as-a-multiplier/).

It's close to a power of 2 (32), which makes multiplication computationally efficient (can be implemented as a bitshift and subtraction: 31*n = (n<<5) - n).

When multiplying by 31, the previous hash value is weighted more heavily than the new character, giving a good balance between positional sensitivity and character values.

31 produces fewer collisions than other small numbers in typical text processing scenarios.




101 is also a prime number, which is ideal for the [modulo operation](https://www.designgurus.io/answers/detail/why-should-hash-functions-use-a-prime-number-modulus) in a hash table.

Using a prime modulus helps distribute hash values more evenly across the hash table.

101 is a reasonable size for a small hash table (determines the number of buckets).

Prime moduli minimize the chance of systematic patterns in the data causing clustering

# Task 3: SHA256 Padding

SHA256 requires messages to be padded to a multiple of 512 bits. 
This function:
1. Reads a file's contents.
2. Appends a `1` bit (`0x80` in hex).
3. Adds `0` bits until the total length is `56 mod 64`.
4. Appends the original message length as a big-endian 64-bit integer.

This padding ensures compatibility with SHA256 hashing.


In [36]:
import struct
import os

def calculate_sha256_padding(file_path):
    """
    Computes the SHA256 padding for a given file.
    
    Args:
    - file_path (str): The path to the input file.
    
    Prints:
    - The padding bytes in hexadecimal.
    """
    if not os.path.exists(file_path):
        print(f"Error: File {file_path} does not exist.")
        return
    
    with open(file_path, 'rb') as f:
        data = f.read()

    print(f"Read {len(data)} bytes from {file_path}")

    original_length = len(data)
    original_bit_length = original_length * 8  # Convert bytes to bits

    # Append '1' bit (0x80 in hex)
    padding = b'\x80'

    # Compute required zero padding
    total_length = original_length + 1
    while (total_length + 8) % 64 != 0:
        padding += b'\x00'
        total_length += 1

    # Append original length in bits as a big-endian 64-bit integer
    padding += struct.pack('>Q', original_bit_length)

    print(f"Padding length: {len(padding)} bytes")
    print("Padding (Hex):", " ".join(f"{byte:02x}" for byte in padding))


Testing the SHA256 Padding Function:
We apply the padding function to 'test.txt`, which contains a short message.


In [37]:
# Run the padding function
calculate_sha256_padding("test.txt")

Read 3 bytes from test.txt
Padding length: 61 bytes
Padding (Hex): 80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18
